# Библиотеки и CUDA

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import torch

In [139]:
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq, WhisperProcessor

In [4]:
print(torch.cuda.is_available())

False


In [5]:
import numpy as np

In [6]:
from datasets import load_dataset

In [7]:
import soundfile as sf

In [8]:
from IPython.display import Audio # штука для работы с аудио

In [9]:
import ast

# Модуль Б. Разработка модели машинного обучения 

_____

В рамках выполнения конкурсного задания необходимо обучить две модели: 

* Преобразование текста в аудиофайл

* Преобразование аудиофайла в текст

Каждая из моделей тесно связано с обработкой *текста* (связного), поэтому необходим токенизатор — это инструмент обработки текста, который разделяет его на отдельные элементы, называемые токенами. Токены могут быть словами, фразами, символами или другими элементами текста, в зависимости от того как работает токенизатор. 

Токенизатор у каждой модели должен быть свой

____

Загружаем данные:

In [10]:
train = load_dataset('csv', data_files = r'..\Module_A\data_selections\train2.csv')
test = load_dataset('csv', data_files = r'..\Module_A\data_selections\test2.csv')
valid = load_dataset('csv', data_files = r'..\Module_A\data_selections\valid2.csv')

In [11]:
print(train)

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'audio_array', 'sr', 'text', 'duration', 'count_words', 'median_pitch', 'db_voice_volume', 'db_noise_level'],
        num_rows: 3881
    })
})


In [12]:
print(test)

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'audio_array', 'sr', 'text', 'duration', 'count_words', 'median_pitch', 'db_voice_volume', 'db_noise_level'],
        num_rows: 685
    })
})


In [13]:
print(valid)

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'audio_array', 'sr', 'text', 'duration', 'count_words', 'median_pitch', 'db_voice_volume', 'db_noise_level'],
        num_rows: 806
    })
})


In [14]:
print(np.array(train['train']['audio_array'][1]), type(np.array(train['train']['audio_array'][1])))

[0.         0.         0.         ... 0.00332642 0.00204468 0.00247192] <class 'numpy.ndarray'>


_____

другие данные:

In [115]:
ds = load_dataset('bond005/sberdevices_golos_10h_crowd', cache_dir = 'old/Module_A/data_sber')

Generating test split: 100%|██████████| 9994/9994 [00:33<00:00, 299.53 examples/s]


In [116]:
print(ds)

DatasetDict({
    train: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 7993
    })
    validation: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 793
    })
    test: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 9994
    })
})


In [119]:
print(type(ds['train']['audio'][0]['array']))

<class 'numpy.ndarray'>


In [178]:
print(ds['test'][467]['transcription'])

понимаешь


In [193]:

example = ds['test'][156]

print(example["audio"]["array"] is None)           # False → всё ок
print(example["audio"]["sampling_rate"] is None)   # False
print(example["transcription"] is None)                 # False

False
False
False


In [195]:
# в отдельные переменные помещаем всю информацию о звуке и весь текст
test_audio = ds['test']['audio'] # аудио
test_txt = ds['test']['transcription'] # текст
print(len(test_audio), len(test_txt), sep = '\n') # выводим кол-во элементов в данных

9994
9994


In [196]:
# списки для хранения данных
arrays = [] # массивы со звуком
sr = [] # частота дискретизации
txt = [] # текст

In [198]:
from tqdm import tqdm

In [200]:
%%time
# проходимся по всем данным
for i in tqdm(range(len(test_audio)), desc='Preproc..', unit='sample'):
    arrays.append(test_audio[i]['array']) # добавляем в соотв. список массив со звуком
    sr.append(test_audio[i]['sampling_rate']) # добавляем в соотв. список значение частоты
    txt.append(test_txt[i]) # добавляем в соотв. список текст

Preproc..: 100%|██████████| 9994/9994 [00:00<00:00, 472335.36sample/s]

CPU times: total: 31.2 ms
Wall time: 30.3 ms


In [201]:
import pandas as pd

In [202]:
df_sb = pd.DataFrame() # создаём датафрейм
df_sb['audio_array'] = arrays # заполняем колонку с массивами
df_sb['sr'] = sr # заполняем колонку с частотой
df_sb['text'] = txt # заполняем колонку с текстом
print(df_sb.shape) # выводим размер дф
df_sb.head() # выводим первые 5 строчек дф

(9994, 3)


,audio_array,sr,text
0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",16000,шестьдесят тысяч тенге сколько будет стоить
1,"[0.0, 0.0, 0.0, -3.0517578125e-05, -3.05175781...",16000,покажи мне на смотрешке телеканал синергия тв
2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",16000,заказать яблоки зеленые
3,"[0.000885009765625, 0.000457763671875, 0.00015...",16000,алиса закажи килограммовый торт графские разва...
4,"[-0.0001220703125, -0.0001220703125, -0.000122...",16000,ищи телеканал про бизнес на тиви


In [203]:
df_sb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   audio_array  9994 non-null   object
 1   sr           9994 non-null   int64 
 2   text         9896 non-null   object
dtypes: int64(1), object(2)
memory usage: 234.4+ KB


In [204]:
df_sb.isnull().sum()

audio_array     0
sr              0
text           98
dtype: int64

In [205]:
df_sb[df_sb.isnull().any(axis=1)]

,audio_array,sr,text
166,"[6.103515625e-05, 3.0517578125e-05, 3.05175781...",16000,None
181,"[-9.1552734375e-05, -0.0001220703125, -0.00012...",16000,None
306,"[-9.1552734375e-05, -9.1552734375e-05, -6.1035...",16000,None
343,"[9.1552734375e-05, 6.103515625e-05, 6.10351562...",16000,None
398,"[0.00018310546875, 0.000213623046875, 0.000213...",16000,None
...,...,...,...
9535,"[-9.1552734375e-05, -9.1552734375e-05, -9.1552...",16000,None
9599,"[-0.000335693359375, -0.00030517578125, -0.000...",16000,None
9617,"[-0.00164794921875, -0.001617431640625, -0.001...",16000,None
9741,"[0.000244140625, 0.000244140625, 0.00021362304...",16000,None


In [206]:
def has_no_missing_fields(example):
    return example["transcription"] is not None and example["audio"]["array"] is not None

ds['test'] = ds["test"].filter(has_no_missing_fields)

Filter: 100%|██████████| 9994/9994 [00:05<00:00, 1687.83 examples/s]


In [207]:
ds

DatasetDict({
    train: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 7993
    })
    validation: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 793
    })
    test: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 9896
    })
})

### **Text-to-audio**

* Модель котора преобразует текст в звук (логичною....дап....ага..)

Модель whisper, веса - small

In [208]:
processor = WhisperProcessor.from_pretrained('openai/whisper-small', language = 'russian', task="transcribe")
model = AutoModelForSpeechSeq2Seq.from_pretrained('openai/whisper-small')

In [ ]:
audio_sample = ds['train']['audio'][0]['array']
print(type(audio_sample))

print(audio_sample)
audio_sr = ds['train']['audio'][0]['sampling_rate']

<class 'numpy.ndarray'>
[ 3.05175781e-05  3.05175781e-05  0.00000000e+00 ... -1.09863281e-03
 -7.93457031e-04 -1.52587891e-04]


In [122]:
input_features = processor(
    audio_sample,
    sampling_rate=audio_sr,
    return_tensors="pt"
).input_features
input_features = input_features.to("cpu", dtype=torch.float32)

forced_decoder_ids = processor.get_decoder_prompt_ids(language="ru", task="transcribe")

predicted_ids = model.generate(input_features, cache_implementation="static", forced_decoder_ids=forced_decoder_ids)
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
transcription[0]

' 16 часть, сезона 5 сериала, лемма нестики, 33 несчастья.'

In [123]:
Audio(audio_sample, rate=16000)

____

токенизатор:

In [124]:
from transformers import WhisperTokenizer

In [150]:
tokenizer = WhisperTokenizer.from_pretrained('openai/whisper-small', language = "russian", task = 'transcribe')

In [209]:
input_str = ds["train"][0]["transcription"]
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")


Input:                 шестнадцатая часть сезона пять сериала лемони сникет тридцать три несчастья
Decoded w/ special:    <|startoftranscript|><|ru|><|transcribe|><|notimestamps|>шестнадцатая часть сезона пять сериала лемони сникет тридцать три несчастья<|endoftext|>
Decoded w/out special: шестнадцатая часть сезона пять сериала лемони сникет тридцать три несчастья
Are equal:             True


Предобработка данных перед обучением:

In [132]:
from datasets import Audio

In [210]:
common_voice = ds.cast_column('audio', Audio(sampling_rate=16000))

обучение..... ыаыаыаы..

In [211]:
# функция для обработки-подготовки датасета к обучению
def prepare_dataset(batch, processor = processor, tokenizer = tokenizer):
    audio = batch['audio']
    batch['input_features'] = processor(audio['array'], sampling_rate = audio['sampling_rate']).input_features[0]

    batch['labels'] = processor.tokenizer(text_target=batch['transcription'], return_tensors="pt").input_ids[0]
    return batch

In [212]:
common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=3)

Map (num_proc=3): 100%|██████████| 9896/9896 [05:28<00:00, 30.10 examples/s]


In [222]:
common_voice.save_to_disk("data/preproc_ds")

Saving the dataset (20/20 shards): 100%|██████████| 9896/9896 [01:54<00:00, 86.63 examples/s] 
